# Data Parsing 

## The following cells will be debugging and development for a data parsing script utilizing pandas and system commands to access, create, and organize files. 

In [1]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import datetime
import glob
import sys
import os
import time



In [2]:
custom_date_parser = lambda x,y,z,a,b: datetime.datetime.strptime(f"{x} {y} {z} {a} {b}", "%Y %m %d %H %M")


In [3]:
data_dir = f"..\\Data\\"


In [4]:
# Create the DataFrame that will be utilized to store all of the parsed data.
BigDF = pd.DataFrame()
# Store all of the files as a list, in order to access it in the loop appropriately.
files = glob.glob(f"../NREL Data/**/*{2020}.csv", recursive=True)
def dateparse(x): return pd.datetime.strptime(x, '%Y %m %d %H %M')

In [5]:
for index in range(0, len(files)):
    start = time.time()
    if len(files) > 2:
        DataFile = files.pop()
        DataFile2 = files.pop()
        Csv_DF = pd.read_csv(DataFile, header=[2], parse_dates={
            'Date': [0, 1, 2, 3, 4]}, date_parser=dateparse)
        Lat_Lon_DF1 = pd.read_csv(DataFile, nrows=1)
        Csv_DF["Longitude"] = Lat_Lon_DF1.Longitude[0]
        Csv_DF["Latitude"] = Lat_Lon_DF1.Latitude[0]
        Csv_DF2 = pd.read_csv(DataFile2, header=[2], parse_dates={
            'Date': [0, 1, 2, 3, 4]}, date_parser=dateparse)
        Lat_Lon_DF2 = pd.read_csv(DataFile2, nrows=1)
        Csv_DF2["Longitude"] = Lat_Lon_DF2.Longitude[0]
        Csv_DF2["Latitude"] = Lat_Lon_DF2.Latitude[0]
        BigDF = BigDF.append([Csv_DF, Csv_DF2])
        # print(BigDF)
    elif len(files) == 1:
        DataFile = files.pop()
        Csv_DF = pd.read_csv(DataFile, header=[2], parse_dates={
            'Date': [0, 1, 2, 3, 4]}, date_parser=dateparse)
        Lat_Lon_DF1 = pd.read_csv(DataFile, nrows=1)
        Csv_DF["Longitude"] = Lat_Lon_DF1.Longitude[0]
        Csv_DF["Latitude"] = Lat_Lon_DF1.Latitude[0]
        BigDF = BigDF.append([Csv_DF])
        # print(BigDF)

Parsed_Data = BigDF.groupby(
    [pd.Grouper(key="Date", freq='1D'), "Latitude", "Longitude"]).mean()
# Parsed_Data = Parsed_Data.round(2)
print(Parsed_Data)

end = time.time()
print("Elapsed time parsed data: ", (end - start))


                                      GHI         DNI  Wind Speed  \
Date       Latitude Longitude                                       
2020-01-01 17.90    -67.29     218.114583  346.333333    2.484375   
                    -67.27     218.062500  345.937500    2.452083   
                    -67.25     218.020833  345.520833    2.419792   
                    -67.23     217.906250  345.000000    2.387500   
                    -67.21     217.739583  344.479167    2.350000   
...                                   ...         ...         ...   
2020-12-31 18.52    -66.47     199.375000  284.270833    6.078125   
                    -66.45     208.593750  302.895833    6.081250   
                    -66.43     200.531250  285.958333    6.083333   
                    -66.41     192.625000  268.500000    6.081250   
                    -66.39     189.125000  266.708333    6.080208   

                               Temperature  Relative Humidity  
Date       Latitude Longitude         

In [13]:
Values_To_Parse = ["DNI", "GHI", "Relative Humidity", "Wind Speed", "Temperature"]
for name in Values_To_Parse:
    start = time.time()
    if not os.path.isdir(data_dir+"2020"+name):
        os.makedirs(data_dir+"2020"+name)
    else:
        continue
    end = time.time()
    print("Elapsed time making directory: ", (end - start))


Elapsed time making directory:  0.0010056495666503906
Elapsed time making directory:  0.0009944438934326172
Elapsed time making directory:  0.0009930133819580078
Elapsed time making directory:  0.0
Elapsed time making directory:  0.000997304916381836


In [19]:
Values_To_Parse = ["DNI"]
expected_date = datetime.date(int(2020), 1, 1)
day_offset = datetime.timedelta(days=1)
for Value in Values_To_Parse:
    start = time.time()
    Data = pd.DataFrame(columns=['value', 'latitude', 'longitude'])
    for rowIndex, row in Parsed_Data.iterrows():
        if rowIndex[0].date() == expected_date:
            Data = Data.append({'value': row[Value], 'latitude':
                                rowIndex[1], 'longitude': rowIndex[2]}, ignore_index=True)
        elif expected_date.year > rowIndex[0].date().year and expected_date.day >= 2:
            expected_date = datetime.date(int(2020), 1, 1)
            continue
        else:
            Data.to_csv(
                path_or_buf=f"{data_dir}{rowIndex[0].strftime('%Y')}{Value}/{Value}{expected_date.strftime('%Y%j')}.csv",
                header=True, index=False, sep=',', mode='w', encoding='utf-8')
            Data.drop(index=Data.index, inplace=True)
            expected_date = expected_date+day_offset
    print(Data)


           value  latitude  longitude
0     296.229167     17.90     -67.27
1     277.593750     17.90     -67.25
2     288.666667     17.90     -67.23
3     303.312500     17.90     -67.21
4     299.833333     17.90     -67.19
...          ...       ...        ...
2875  284.270833     18.52     -66.47
2876  302.895833     18.52     -66.45
2877  285.958333     18.52     -66.43
2878  268.500000     18.52     -66.41
2879  266.708333     18.52     -66.39

[2880 rows x 3 columns]


In [ ]:
expected_date = datetime.date(int(2020), 1, 1)
Values_To_Parse = ["DNI", "GHI",
                   "Relative Humidity", "Temperature", "Wind Speed"]
day_offset = datetime.timedelta(days=1)
for Value in Values_To_Parse:
    start = time.time()
    Data = pd.DataFrame(columns=['value', 'latitude', 'longitude'])
    for rowIndex, row in Parsed_Data.iterrows():
        # print(f"rowIndex[0] = {rowIndex[0]}")
        # print(f"rowIndex[1] = {rowIndex[1]}")
        # print(f"rowIndex[2] = {rowIndex[2]}")
        # print(f"row = {row}")
        if rowIndex[0].date() == expected_date:
            Data = Data.append({'value': row[Value], 'latitude':
                                rowIndex[1], 'longitude': rowIndex[2]}, ignore_index=True)
            # print(Data)
        elif expected_date.year > rowIndex[0].date().year and expected_date.day >= 2:
            expected_date = datetime.date(int(2020), 1, 1)
            # print(f"Expected date: {expected_date}")
            continue
        else:
            Data.to_csv(
                path_or_buf=f"{data_dir}{rowIndex[0].strftime('%Y')}{Value}/{Value}{expected_date.strftime('%Y%j')}.csv",
                header=False, index=False, sep='')
            Data.drop(index=Data.index, inplace=True)
            expected_date = expected_date+day_offset
            # print(f"This is the expected date after the offset: {expected_date}")
    print(Data)
